In [1]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/dari-pasentran-ke-istana.gz
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/turki.gz
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/salina.gz

In [2]:
# !tar -xf dari-pasentran-ke-istana.gz
# !tar -xf turki.gz
# !tar -xf salina.gz

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

torch_dtype = torch.bfloat16

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_flash_attention_2 = True
)
model.to('cuda')

You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperFlashAttention2(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, b

In [6]:
processor = AutoProcessor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=1,
    return_timestamps=True,
    torch_dtype=torch.bfloat16,
    device='cuda',
)

In [8]:
from glob import glob

files = glob('*/output-wav*/*.wav')
files = files[:len(files) // 2]
len(files)

12184

In [ ]:
from tqdm import tqdm

results = {}
for f in tqdm(files):
    result = pipe(f)
    results[f] = result

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████▊  | 11922/12184 [1:50:44<02:12,  1.98it/s]